In [1]:
from collections import deque, namedtuple


# we'll use infinity as a default distance to nodes.
inf = float('inf')
Edge = namedtuple('Edge', 'start, end, cost')


def make_edge(start, end, cost=1):
  return Edge(start, end, cost)


class Graph:
    def __init__(self, edges):
        # let's check that the data is right
        wrong_edges = [i for i in edges if len(i) not in [2, 3]]
        if wrong_edges:
            raise ValueError('Wrong edges data: {}'.format(wrong_edges))

        self.edges = [make_edge(*edge) for edge in edges]

    @property
    def vertices(self):
        return set(
            sum(
                ([edge.start, edge.end] for edge in self.edges), []
            )
        )

    def get_node_pairs(self, n1, n2, both_ends=True):
        if both_ends:
            node_pairs = [[n1, n2], [n2, n1]]
        else:
            node_pairs = [[n1, n2]]
        return node_pairs

    def remove_edge(self, n1, n2, both_ends=True):
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        edges = self.edges[:]
        for edge in edges:
            if [edge.start, edge.end] in node_pairs:
                self.edges.remove(edge)

    def add_edge(self, n1, n2, cost=1, both_ends=True):
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        for edge in self.edges:
            if [edge.start, edge.end] in node_pairs:
                return ValueError('Edge {} {} already exists'.format(n1, n2))

        self.edges.append(Edge(start=n1, end=n2, cost=cost))
        if both_ends:
            self.edges.append(Edge(start=n2, end=n1, cost=cost))

    @property
    def neighbours(self):
        neighbours = {vertex: set() for vertex in self.vertices}
        for edge in self.edges:
            neighbours[edge.start].add((edge.end, edge.cost))

        return neighbours

    def dijkstra(self, source, dest):
        assert source in self.vertices, 'Such source node doesn\'t exist'
        distances = {vertex: inf for vertex in self.vertices}
        previous_vertices = {
            vertex: None for vertex in self.vertices
        }
        distances[source] = 0
        vertices = self.vertices.copy()

        while vertices:
            current_vertex = min(
                vertices, key=lambda vertex: distances[vertex])
            vertices.remove(current_vertex)
            if distances[current_vertex] == inf:
                break
            for neighbour, cost in self.neighbours[current_vertex]:
                alternative_route = distances[current_vertex] + cost
                if alternative_route < distances[neighbour]:
                    distances[neighbour] = alternative_route
                    previous_vertices[neighbour] = current_vertex

        path, current_vertex = deque(), dest
        while previous_vertices[current_vertex] is not None:
            path.appendleft(current_vertex)
            current_vertex = previous_vertices[current_vertex]
        if path:
            path.appendleft(current_vertex)
        return path


graph = Graph([
    ("a", "b", 7),  ("a", "c", 9),  ("a", "f", 14), ("b", "c", 10),
    ("b", "d", 15), ("c", "d", 11), ("c", "f", 2),  ("d", "e", 6),
    ("e", "f", 9)])



In [2]:
print(graph.dijkstra("b", "f"))

deque(['b', 'c', 'f'])


In [3]:
import pandas as pd

In [55]:
covid19DF = pd.read_csv("./Data/04-12-2020.csv", usecols = ["Admin2", "Province_State", "Country_Region",
                                                           "Confirmed"])

In [56]:
covid19DF_us = covid19DF.loc[covid19DF['Country_Region'] == 'US']
covid19DF_georgia = covid19DF_us.loc[covid19DF['Province_State'] == 'Georgia']
covid19DF_georgia

,Admin2,Province_State,Country_Region,Confirmed
65,Appling,Georgia,US,9
87,Atkinson,Georgia,US,2
99,Bacon,Georgia,US,14
101,Baker,Georgia,US,14
103,Baldwin,Georgia,US,52
...,...,...,...,...
2631,Whitfield,Georgia,US,24
2638,Wilcox,Georgia,US,22
2639,Wilkes,Georgia,US,6
2642,Wilkinson,Georgia,US,11


In [57]:
census_regions = pd.read_csv("./Data/census_georgia.csv")

In [58]:
for county in census_regions['Admin2']:
    census_regions.Admin2[census_regions.Admin2 == county] = county.replace(".", '').split()[0]

census_regions

,Admin2,2019
0,Appling,"18,386"
1,Atkinson,"8,165"
2,Bacon,"11,164"
3,Baker,"3,038"
4,Baldwin,"44,890"
...,...,...
154,Whitfield,"104,628"
155,Wilcox,"8,635"
156,Wilkes,"9,777"
157,Wilkinson,"8,954"


In [66]:
dijkstraDF = covid19DF_georgia.merge(census_regions, on="Admin2", how = 'inner')

In [68]:
dijkstraDF = dijkstraDF.drop('Province_State', 1).drop('Country_Region', 1)
dijkstraDF

,Admin2,Confirmed,2019
0,Appling,9,"18,386"
1,Atkinson,2,"8,165"
2,Bacon,14,"11,164"
3,Baker,14,"3,038"
4,Baldwin,52,"44,890"
...,...,...,...
149,Whitfield,24,"104,628"
150,Wilcox,22,"8,635"
151,Wilkes,6,"9,777"
152,Wilkinson,11,"8,954"


In [70]:
dijkstraDF.to_csv('dijkstraInput.csv')